## Number theoretical tools: The Skolem polynomials and its inverse

In [186]:
from math import comb

def combinadics_from_index(N, k):
    """
    This function converts a natural number N as sequences of k-combinations. 
    This is referred to as the combinatorial number system of degree k (for some positive integer k).
    
    See https://planetcalc.com/8592/
    """
    combinadics = [0] * k
    
    c_start = max(N, k)
    
    for j in range(k, 0, -1):
    
        # the loop over c can be written more efficiently by using the bisection method
        for c in range(c_start, -1, -1):
            
            q = comb(c, j)
            
            if q <= N:
                break
                print(c)
                
        c_start = c - 1
        N -= q
        combinadics[k - j] = c
    
    return combinadics

def index_from_combinadics(combinadics):
    """
    This function returns index from the combinadics.
    
    Note:
        assert index_from_combinadics(combinadics_from_index(N, k)) == N
    """
    return sum(
        comb(c, j + 1) for j, c in enumerate(reversed(combinadics))
    )

def skolem(ntuple):
    """
    Evaluate the Skolem polynomial from tuple
    """
    return sum(
        comb(sum(ntuple[:k]) + k - 1, k) for k in range(1, len(ntuple) + 1)
    )

def tuple_from_skolem(S, k):
    "The inverse of the Skolem polynomials"
    
    combinadics = combinadics_from_index(S, k)
    
    assert index_from_combinadics(combinadics) == S

    ntuple = [0] * k
    
    ntuple[0] = combinadics[-1]
    
    for j in range(1, k):
        ntuple[j] = combinadics[k - j - 1] - j - sum(ntuple[:j])
    
    return ntuple

In [187]:
tuple_from_skolem(skolem([0, 0, 1]), 3)

[0, 0, 1]

In [193]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh_tridiagonal, eigvals

U = 0.
Δt = 0.1

N = 3

# the cut off of Skolem index
S_max = max(skolem([N, 0, 0]), skolem([0, N, 0]), skolem([0, 0, N]))


#skolem_123 = np.arange(S_max + 1)
skolem_123 = np.arange(skolem([0, 0, N]), skolem([N, 0, 0]) + 1)
                              
n1, n2, n3 = np.array(
    [tuple_from_skolem(S, 3) for S in skolem_123]
).T

skolem_321 = np.array([
    skolem(_) for _ in zip(n3, n2, n1)
])

# check whether the array is continuous -- Great
#assert np.allclose(np.diff(np.sort(skolem_321)), 1)

"""
skolem_312 = np.array([
    skolem(_) for _ in zip(n3, n1, n2)
])

# check whether the array is continuous -- Great
assert np.allclose(np.diff(np.sort(skolem_312)), 1)
"""


'\nskolem_312 = np.array([\n    skolem(_) for _ in zip(n3, n1, n2)\n])\n\n# check whether the array is continuous -- Great\nassert np.allclose(np.diff(np.sort(skolem_312)), 1)\n'

In [194]:
n1, n2, n3

(array([0, 0, 1, 0, 1, 2, 0, 1, 2, 3]),
 array([0, 1, 0, 2, 1, 0, 3, 2, 1, 0]),
 array([3, 2, 2, 1, 1, 1, 0, 0, 0, 0]))

In [195]:
((n1 + 1) * n2)[:-1]

array([0, 1, 0, 2, 2, 0, 3, 4, 3])

In [196]:

"""
print(E_123)

# The propagator
U_123 = v @ np.diag(np.exp(-Δt * E_123)) @ v.conj().T


E_321, v = eigh_tridiagonal(
    U * n3 * (n3 - 1) + U * n2 * (n2 - 1), 
    -np.sqrt((n3 + 1) * n2)[1:],
    # eigvals_only = True,
    #select='i',
    #select_range=(0, N ** 2)
)

# The propagator
U_321 = v @ np.diag(np.exp(-Δt * E_321)) @ v.conj().T


Permutation = np.zeros_like(v)
Permutation[skolem_123, skolem_321] = 1

Propagator = Permutation.T @ U_321 @ Permutation @ U_123
"""

E_123, v = eigh_tridiagonal(
    U * n1 * (n1 - 1),
    -np.sqrt((n1 + 1) * n2)[:-1],
    # eigvals_only = True,
    #select='i',
    #select_range=(0, N ** 2)
)

E_123.size

10

In [197]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import boson_basis_1d # Hilbert space boson basis

##### define model parameters #####
L = 6 # system size
J = 1.0 # hopping
U = np.sqrt(2.0) # interaction

##### construct Bose-Hubbard Hamiltonian #####
# define boson basis with 3 states per site L bosons in the lattice
basis = boson_basis_1d(L,Nb=L) # full boson basis
print(basis)

ModuleNotFoundError: No module named 'quspin'

In [198]:
!pip install quspin

ERROR: Could not find a version that satisfies the requirement quspin (from versions: none)
ERROR: No matching distribution found for quspin


In [199]:
#a1 = tensor(tensor(qeye(N), qeye(N)), destroy(N))
a1 = tensor(qeye(N), tensor(qeye(N), destroy(N)))

#a2 = tensor(tensor(qeye(N), destroy(N)), qeye(N))
a2 = tensor(qeye(N), tensor(destroy(N), qeye(N)))

a3 = tensor(destroy(N), tensor(qeye(N), qeye(N)))

num_1 = a1.dag() * a1
num_2 = a2.dag() * a2
num_3 = a3.dag() * a3

E1_qutip = (
    -(a1.dag() * a2 + a2.dag() * a1) + U * num_1 * (num_1 - 1)
).eigenenergies()

In [200]:
E2_qutip = (
    -(a2.dag() * a3 + a3.dag() * a2) + U * num_3 * (num_3 - 1)
).eigenenergies()

In [201]:
np.allclose(E1_qutip, E2_qutip)

True

In [205]:
E1_qutip

array([-2.00000000e+00, -2.00000000e+00, -2.00000000e+00, -2.00000000e+00,
       -2.00000000e+00, -2.00000000e+00, -1.00000000e+00, -1.00000000e+00,
       -1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.38774531e-16,
        2.38774531e-16,  2.38774531e-16,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00,  2.00000000e+00])

In [203]:
E1_qutip, E_123

(array([-2.00000000e+00, -2.00000000e+00, -2.00000000e+00, -2.00000000e+00,
        -2.00000000e+00, -2.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.38774531e-16,
         2.38774531e-16,  2.38774531e-16,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
         2.00000000e+00,  2.00000000e+00,  2.00000000e+00]),
 array([-3.00000000e+00, -2.00000000e+00, -1.00000000e+00, -1.00000000e+00,
         0.00000000e+00,  2.22044605e-15,  1.00000000e+00,  1.00000000e+00,
         2.00000000e+00,  3.00000000e+00]))

In [78]:
p = np.sort(eigvals(Propagator).real)[-1]
-np.log(p) / Δt

-3.3946178162411136

In [75]:
from qutip import tensor, destroy, qeye

def get_U_qutip(N):
    
    a1 = tensor(tensor(destroy(N), qeye(N)), qeye(N))
    a2 = tensor(tensor(qeye(N), destroy(N)), qeye(N))
    a3 = tensor(tensor(qeye(N), qeye(N)), destroy(N))

    n1 = a1.dag() * a1
    n2 = a2.dag() * a2
    n3 = a3.dag() * a3

    H = -(a1.dag() * a2 + a2.dag() * a1  + a2.dag() * a3 + a3.dag() * a2) + \
        U * (n1 * (n1 - 1) + n2 * (n2 - 1) + n3 * (n3 - 1))
    
    print(H.eigenenergies().min())
    #return (-Δt * H).expm().eigenenergies()
    
    U_123 = (-Δt * (
        -(a1.dag() * a2 + a2.dag() * a1) + U * n1 * (n1 - 1)
        
    )).expm()

    U_231 = (-Δt * (
        -( a2.dag() * a3 + a3.dag() * a2) + U * (n2 * (n2 - 1) + n3 * (n3 - 1))
        
    )).expm()

    
    return (U_231 * U_123).eigenenergies()
    
U_qutip = get_U_qutip(N)

-1.414213562373095


In [76]:
-np.log(U_qutip.real.max()) /  Δt

-1.4145080613257948

In [72]:
U_qutip.size

8